# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Data Engineering and Machine Learning Operations in Business** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## <span style='color:#2656a3'> 🗒️ This notebook is divided into the following sections:

1. Load batch data.
2. Predict using model from Model Registry.

## <span style='color:#2656a3'> ⚙️ Import of libraries and packages

First, we'll install the Python packages required for this notebook. We'll use the --quiet command after specifying the names of the libraries to ensure a silent installation process. Then, we'll proceed to import all the necessary libraries.

In [ ]:
# Importing the packages for the needed libraries for the Jupyter notebook
import joblib
import inspect 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import os

#%config InlineBackend.figure_format='retina'
#%matplotlib inline

## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

In [ ]:
# Importing the hopsworks module
import hopsworks

# Logging in to the Hopsworks project
project = hopsworks.login()

# Getting the feature store from the project
fs = project.get_feature_store() 

### <span style='color:#2656a3'> ⚙️ Feature View Retrieval

In [ ]:
# Retrieve the 'electricity_feature_view' feature view
feature_view = fs.get_feature_view(
    name='electricity_feature_view',
    version=1,
)

### <span style='color:#2656a3'> 🗄 Model Registry

In [ ]:
# Retrieve the model registry
mr = project.get_model_registry()

## <span style='color:#2656a3'> 📮 Retrieving model from Model Registry

In [ ]:
# Retrieving the model from the Model Registry
retrieved_model = mr.get_model(
    name="electricity_price_prediction_model", 
    version=1,
)

# Downloading the saved model to a local directory
saved_model_dir = retrieved_model.download()

# Loading the saved XGB model
retrieved_xgboost_model = joblib.load(saved_model_dir + "/dk_electricity_model.pkl")

In [ ]:
# Display the retrieved XGBoost regressor model
retrieved_xgboost_model

## <span style='color:#2656a3'> ✨ Load Batch Data

In [ ]:
import datetime

# Calculating the start date as 5 days ago from the current date
start_date = datetime.datetime.now() - datetime.timedelta(days=5)

# Converting the start date to a timestamp in milliseconds
start_time = int(start_date.timestamp()) * 1000

# Displaying the start date in timestamp format
start_time

In [ ]:
# Initializing batch scoring
feature_view.init_batch_scoring(1)

# Retrieving batch data from the feature view starting from the specified start time
batch_data = feature_view.get_batch_data(
    start_time=start_time,
)

batch_data

In [ ]:
# First we go one back in our directory to access the folder with our functions
%cd ..

# Now we import the functions from the features folder
# This is the functions we have created to generate features for electricity prices and weather measures
from features import electricity_prices, weather_measures 

# We go back into the notebooks folder
%cd notebooks

In [ ]:
# Fetching weather forecast measures for the next 5 days
weather_forecast_df = weather_measures.forecast_weather_measures(
    forecast_length=5
)

In [ ]:
# Read csv file with calender
calender_df = pd.read_csv('https://raw.githubusercontent.com/Camillahannesbo/MLOPs-Assignment-/main/data/calendar_incl_holiday.csv', delimiter=';', usecols=['date', 'type'])

calender_df

In [ ]:
from datetime import datetime, timedelta

# Formatting the date column to 'YYYY-MM-DD' dateformat
calender_df["date"] = calender_df["date"].map(lambda x: datetime.strptime(x, '%d/%m/%Y').strftime("%Y-%m-%d"))

In [ ]:
# Add features to the calender dataframe
calender_df['date_'] = pd.to_datetime(calender_df['date'])
calender_df['day'] = calender_df['date_'].dt.dayofweek
calender_df['month'] = calender_df['date_'].dt.month
calender_df['holiday'] = np.where(calender_df['type'] == 'Not a Workday', 1, 0)

# Drop the columns 'type' and 'date_' to finalize the calender dataframe
calender_df = calender_df.drop(['type','date_'], axis=1)

merged_df = pd.merge(weather_forecast_df, calender_df, how='inner', left_on='date', right_on='date')

In [ ]:
import numpy as np

In [ ]:
# Display the first 5 rows of the batch data
batch_data = merged_df

batch_data.tail()

### <span style="color:#ff5f27;">🤖 Making the predictions</span>

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# # Create a LabelEncoder object
# label_encoder = LabelEncoder()

# # Fit the encoder to the data in the 'city_name' column
# label_encoder.fit(batch_data[['type']])

# # Transform the 'city_name' column data using the fitted encoder
# encoded = label_encoder.transform(batch_data[['type']])

In [ ]:
batch_data

In [ ]:
# # Convert the output of the label encoding to a dense array and concatenate with the original data
# X_batch = pd.concat([batch_data, pd.DataFrame(encoded)], axis=1)

X_batch = batch_data

# Drop columns 'date', 'city_name', 'unix_time' from the DataFrame 'X'
X_batch = X_batch.drop(columns=['date', 'time', 'timestamp'])

# # Rename the newly added column with label-encoded city names to 'city_name_encoded'
# X_batch = X_batch.rename(columns={0: "type_encoded"})

# Displaying the first 5 rows of the modified DataFrame
X_batch.head()

In [ ]:
# Extract the target variable 'dk1_spotpricedkk_kwh' from the batch data
y_batch = X_batch.pop('dk1_spotpricedkk_kwh')

# Displaying the first 5 rows of the modified DataFrame
y_batch.head()

In [ ]:
X_batch

In [ ]:
# Make predictions on the batch data using the retrieved XGBoost regressor model
predictions = retrieved_xgboost_model.predict(X_batch)

# Display the first 5 predictions
predictions[:5]

In [ ]:
label = batch_data["time"]
y_pred = retrieved_xgboost_model.predict(X_batch)

data = {
    'prediction': [y_pred],
    'time': [label],
}

monitor_df = pd.DataFrame(data)
monitor_df

In [ ]:
label = batch_data["time"]
y_pred = retrieved_xgboost_model.predict(X_batch)

data = {
    'prediction': y_pred,
    'time': label,
}

monitor_df = pd.DataFrame(data)
monitor_df

---
## <span style="color:#ff5f27;">👾 Next is creating our Streamlit App?</span>